### using langsmith for debugging, monitoring 


In [48]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true' 
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

### using document loaders to create a knowledge base base

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.moneycontrol.com/news/business/ratan-tata-death-news-live-updates-tata-group-chairman-last-rights-funeral-liveblog-12839261.html")

docs = loader.load()

### creating different chunks by splitting the docs

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)


### creating embeddings for the docs and store it in a vector store

 https://www.rungalileo.io/blog/mastering-rag-how-to-select-an-embedding-model#:~:text=When%20selecting%20an%20embedding%20model,%2C%20and%20Voyager%2C%20among%20others.

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="llama3.2"
)
vector_store = Chroma.from_documents(documents=splits, embedding= embed)

In [ ]:
print(len(vector_store))

### retriever

In [53]:
retriever = vector_store.as_retriever()

###  using ollama and meta3.2 3B

In [54]:
from langchain_ollama import ChatOllama
local_llm = "llama3.2"
llm = ChatOllama(model=local_llm, temperature=0)



In [55]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {question}""")
parser = StrOutputParser()



In [56]:
from langchain_core.runnables import RunnablePassthrough

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



### Question


In [ ]:
rag_chain.invoke("What is Task Decomposition?")

In [ ]:
rag_chain.invoke("Who is Ratan Tata")

In [ ]:
rag_chain.invoke("Which people are mentioned in the article")